In [9]:
#libraries
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx
import _validation_scripts as vs
import openmatrix as omx
import geopandas as gpd
from shapely.geometry import Point
import math
import glob

# inputs
df_obs_path = r"data/5-assignhwy/observed-volumes.csv"
df_mod_path = r"data/5-assignhwy/WFv910_BY_2019_Summary_SEGID.csv"
omx_path = r"../../../_large_files/v910/5_FinalNetSkims/"
geojson_file = r'data/5-assignhwy/tazNew.geojson'

# read segment shapefile into spatial enabled dataframe
dirSegShp = r'data/5-assignhwy/seg_shp/WFv910_Segments.shp'
segShp = gpd.read_file(dirSegShp)
segShp.head()
segShp = segShp[['SEGID','geometry']]

In [10]:
#read in model/observed data
df_obs = pd.read_csv(df_obs_path)
df_mod = pd.read_csv(df_mod_path)
df_ft = df_mod[['SEGID','FTCLASS']]
df_fc = df_obs[['SEGID','Valid_FC']]

# prep comparison dfs for ojs volume summary comparisons
df_mod1 = df_mod.copy()
df_mod1['Mod_Car'] = df_mod1['DY_Vol_PC'] + df_mod1['DY_Vol_LT']
df_mod1 = df_mod1.rename(columns={'DY_Vol':'Mod_AWDT','DY_Vol_MD':'Mod_MD','DY_Vol_HV':'Mod_HV'})
df_mod1 = df_mod1[['SEGID','CO_FIPS','DISTANCE','Mod_AWDT','Mod_Car','Mod_MD','Mod_HV']]

df_obs1 = df_obs[['SEGID', 'Obs_AWDT', 'Obs_Car', 'Obs_MD', 'Obs_HV']]
allveh = pd.merge(df_mod1,df_obs1,on='SEGID',how='left')
allveh = pd.merge(allveh,df_fc, on='SEGID', how='left')

In [11]:
# prep map comparison of volumes
df_obs[['data']] = 'Observed'
df_mod[['data']] = 'Modeled'

# observed
df_obs = df_obs[['SEGID','data','Obs_AWDT','Obs_Car','Obs_MD','Obs_HV']].rename(columns={'Obs_AWDT':'Total','Obs_Car':'Car','Obs_MD':'MD','Obs_HV':'HV'})

#modeled
df_mod['Car'] = df_mod['DY_Vol_PC'] + df_mod['DY_Vol_LT']
df_mod = df_mod.rename(columns={'DY_Vol':'Total','DY_Vol_MD':'MD','DY_Vol_HV':'HV'})
df_mod = df_mod[['SEGID','data','Total', 'Car','MD', 'HV']]

In [12]:
# create one large dataframe before creating maps
dfSegSum = pd.concat([df_mod,df_obs],ignore_index=True)
dfSegSum = pd.merge(dfSegSum,df_ft,on='SEGID',how='left')

In [42]:
def plot_volume_diff(segSum, varOption, segShp):
    scenario1 = 'Modeled'
    scenario2 = 'Observed'

    lstScenario = list(set(segSum.data.tolist()))
    segSum = segSum[['SEGID', 'FTCLASS', 'data', varOption]]
    dfSegSum1 = segSum.query('data == @scenario1')
    dfSegSum2 = segSum.query('data == @scenario2')

    dfSegSum2[varOption] *= -1
    dfSegSumDiff = pd.merge(dfSegSum1, dfSegSum2, on= ['SEGID'], how='left')
    dfSegSumDiff['diff'] = dfSegSumDiff[varOption + '_x'] + dfSegSumDiff[varOption + '_y']
    dfSegSumDiff['FTCLASS'] = dfSegSumDiff['FTCLASS_x']

    sdfSegSumDiff = segShp.merge(dfSegSumDiff, on = 'SEGID')

    conditions = [
        (sdfSegSumDiff['diff'].lt(-10000)),
        (sdfSegSumDiff['diff'].ge(-10000) & sdfSegSumDiff['diff'].lt(-3000)),
        (sdfSegSumDiff['diff'].ge(-3000) & sdfSegSumDiff['diff'].lt(-1000)),
        (sdfSegSumDiff['diff'].ge(-1000) & sdfSegSumDiff['diff'].lt(1000)),
        (sdfSegSumDiff['diff'].ge(1000) & sdfSegSumDiff['diff'].lt(3000)),
        (sdfSegSumDiff['diff'].ge(3000) & sdfSegSumDiff['diff'].le(10000)),
        (sdfSegSumDiff['diff'].gt(10000)),
    ]
    choices = [2.4,2,1.7,1.7,1.7,2,2.4]
    sdfSegSumDiff["lw"] = np.select(conditions, choices)
    sdfSegSumDiff['lwf'] = np.where(sdfSegSumDiff['FTCLASS'] == 'Freeway', sdfSegSumDiff['lw'], sdfSegSumDiff['lw'] - 1.6)

    # Create the figure and axis
    fig, ax = plt.subplots()

    # Check and set CRS if necessary
    if sdfSegSumDiff.crs is None:
        # Assuming your original data is in EPSG:4326 (WGS84)
        sdfSegSumDiff.set_crs(epsg=26912, inplace=True)

    # Check if we need to reproject to Web Mercator
    if sdfSegSumDiff.crs.to_string() != 'EPSG:3857':
        sdfSegSumDiff = sdfSegSumDiff.to_crs(epsg=3857)
    
    bin1 = [-15000, -7500, -2500, 0, 2500, 7500, 15000]
    bin2 = [-5000, -1500, -500, 0, 500, 1500, 5000]
    
    if varOption=='Total':
        bin = bin1
        titleName = 'All Vehicles'
        tick_labels = ['<-15000', '-7500', '-2500', '0', '2500', '7500', '>15000'] 
    else:
        bin = bin2
        titleName = varOption
        tick_labels = ['<-5000', '-1500', '-500', '0', '500', '1500', '>5000'] 
    
    # Plot your geospatial data
    sdfSegSumDiff.plot(
        column='diff', 
        cmap='RdBu_r', 
        scheme="userdefined", 
        legend=True, 
        classification_kwds=dict(bins=bin),
        linewidth=sdfSegSumDiff['lwf'], 
        ax=ax,
        antialiased=False
    )

    # Add basemap using contextily with OpenStreetMap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.PositronNoLabels, alpha=1)

    # Adjust the margins and axis
    ax.margins(0.1)
    ax.axis('off')

    # Adjust the x-axis limits to cut off the right side of the map
    xlim = ax.get_xlim()  # Get current x-axis limits
    cutoff_value = xlim[1] - 65000  # Define how much you want to cut off (adjust value as needed)
    ax.set_xlim(xlim[0], cutoff_value)  # Set new x-axis limits


    # Adjust legend size
    leg = ax.get_legend()  # Get the current legend
    leg.set_bbox_to_anchor((1, 1))  # Move the legend outside the plot area if necessary
    leg.set_title('Difference Scale', prop={'size': 12})  # Adjust the title size
    for text in leg.get_texts():
        text.set_fontsize(12)  # Adjust the size of the legend text

    # Set the title using the varOption variable
    ax.set_title(f'Difference in Volume: {titleName}', loc='center', fontsize=14, pad=20)


    # Show the plot
    plt.rcParams["figure.figsize"]=6,12
    plt.tight_layout()
    plt.savefig(f'_pictures/vol-diff-{varOption}.png', bbox_inches='tight', dp=12000)
    plt.close(fig)

In [46]:
plot_volume_diff(dfSegSum,'Car', segShp)
#vs.plot_volume_diff(dfSegSum,'Total', segShp) # not sure why i need to run this twice...
#vs.plot_volume_diff(dfSegSum,'Car', segShp)
#vs.plot_volume_diff(dfSegSum,'MD', segShp)
#vs.plot_volume_diff(dfSegSum,'HV', segShp)

C:\Users\cday\AppData\Local\Temp\ipykernel_536\3104687453.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSegSum2[varOption] *= -1
C:\Users\cday\AppData\Local\Temp\ipykernel_536\3104687453.py:93: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "dp" which is no longer supported as of 3.3 and will become an error two minor releases later
  plt.savefig(f'_pictures/vol-diff-{varOption}.png', bbox_inches='tight', dp=12000)
